# AXA coding challenge
Data:
1. Citibike: https://s3.amazonaws.com/tripdata/index.html
2. NYPD:  https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/about_data

In [3]:
# Install packages (only once)
#!pip install selenium webdriver-manager

# Import modules
import os # basic
import datetime
import zipfile
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import dask.dataframe as dd

from selenium import webdriver # for downloading files automatically
from selenium.webdriver.chrome.service import Service

current_dir = os.getcwd() # current dir
print('Current directory: ' + current_dir)
extract_dir = current_dir + '/raw_data/bike-tripdata'  # directory where extracted files from 1. will be saved
cleaned_dir = current_dir + '/clean_data/' # directory where cleaned and concatenated df will be saved

#pd.options.display.float_format = '{:.4f}'.format # set pd output to 2 decimals
#pd.reset_option('display.float_format')
warnings.filterwarnings("ignore")

Current directory: C:\Users\Hanna\sciebo\AXA_coding-challenge


In [4]:
# Functions

# to download files from an url
def download_files(url, save_path):
    response = requests.get(url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
    print(f"Downloaded {save_path}")
    
# to clean column names
def clean_column_names(df, column_mapping=None):
    # strip whitespace, convert to lowercase, and replace spaces with underscores
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
    
    # apply manual column mapping if specified
    if column_mapping:
        df.rename(columns=column_mapping, inplace=True)
    
    return df

# check unique column names across all .csv files in list_files 
def list_unique_col_names(list_files):
    unique_column_names = []
    for csv_file in list_files:
        file_path = os.path.join(extract_dir, csv_file)
        df = pd.read_csv(file_path, nrows=1)
        #print(df.columns) # visual check
        [unique_column_names.append(col) for col in df.columns if col not in unique_column_names]
    unique_column_names.sort()
    
    return unique_column_names

## Download Citibike data automatically from url

In [7]:
url = "https://s3.amazonaws.com/tripdata/index.html" # url to data files
driver_path = 'C:/Drivers/chromedriver-win64_128/chromedriver.exe' # Chrome driver for web interaction, needed by selenium - must match Chrome version

# Download files
service = Service(driver_path) # initialize the Chrome driver
driver = webdriver.Chrome(service=service)
driver.get(url) # navigate to website
time.sleep(5)  # give the page time to load the dynamic content
html = driver.page_source # get the page source after JavaScript has executed
soup = BeautifulSoup(html, 'html.parser') # parse the HTML

# find all .zip links
file_links = []
for link in soup.find_all('a', href=True):
    if link['href'].endswith('.zip'): # on this website, files are .zip format
        file_links.append(link['href'])
print(file_links[:2]) # check if the file paths are retrieved correctly by printing a few

driver.quit() # close the browser

if not os.path.exists(current_dir+'/downloads'): # directory to save the downloaded files
    os.makedirs(current_dir+'/downloads')

for file_link in file_links: # loop through all the zip links and download them
    filename = os.path.join(current_dir+'/downloads', os.path.basename(file_link))
    
    if not file_link.startswith('http'): # if the link is relative, make it an absolute URL by appending the base URL
        file_link = url + file_link

    download_files(file_link, filename) # download the file

<Response [200]>
[]


## Unzip & reorganize files

In [ ]:
# - alternatively (instead of next cell), unzip first and then reorganize files

# Unzip files  
# zip_dir = current_dir+'/downloads' # directory containing the zip files
# extract_dir = current_dir+'/data' # directory where extracted files will be saved

# for filename in os.listdir(zip_dir): # loop through all files in the directory
#     if filename.endswith('.zip') :
#         zip_file_path = os.path.join(zip_dir, filename)
#         new_file_path = extract_dir + '/' + filename[:-4] + '.csv' # remove '.zip' and subfolders from the target path name
#         os.makedirs(new_file_path, exist_ok=True)  # create the directory if it doesn't exist

#         with zipfile.ZipFile(zip_file_path, 'r') as zip_ref: # extract the zip file
#             for member in zip_ref.namelist():
#                 if '_MACOSX' not in member: # skip any file or folder inside "_MACOSX" (for MAC computers, not needed)
#                     zip_ref.extract(member, new_file_path) # extract to the specified directory

#             print(f'Extracted: {member} to {new_file_path}')


# # Move  files from subfolders in subfolders to 1 folder

# import shutil

# source_dir = current_dir + '/data'
# destination_dir = current_dir + '/data_test'
# os.makedirs(destination_dir, exist_ok=True)

# for root, dirs, files in os.walk(source_dir):
#     for file in files:
#         if file.endswith('.csv') and not file.startswith('.'): # select .csv files, skip files starting with '.' 
#             if '_MACOSX' in root:
#                 continue  # skip this directory and its contents, for MAC

#             source_file = os.path.join(root, file)
#             destination_file = os.path.join(destination_dir, file)
            
#             shutil.move(source_file, destination_file) # or shutil.copy
#             print(f"Moved: {source_file} -> {destination_file}")


In [95]:
# Unzip files & reorganize simultaneously
zip_dir = current_dir + '/downloads'  # directory containing the zip files
extract_dir = current_dir + '/data/bike-tripdata'  # directory where extracted files will be saved

os.makedirs(extract_dir, exist_ok=True)  # create the directory if it doesn't exist

for filename in os.listdir(zip_dir):  # loop through all files in the directory
    if filename.endswith('.zip'):
        zip_file_path = os.path.join(zip_dir, filename)

        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:  # extract the zip file
            for member in zip_ref.namelist():
                # skip any file or folder inside "_MACOSX" (for MAC computers, not needed), and files that do not end with .csv
                if '_MACOSX' not in member and member.endswith('.csv'):  
                    # get only the base name of the file (ignore folder structure in zip)
                    base_member = os.path.basename(member)
                    target_path = os.path.join(extract_dir, base_member)
                    
                    with zip_ref.open(member) as source, open(target_path, 'wb') as target:
                        target.write(source.read())  # write the extracted content to the single folder

                    print(f'Extracted {base_member}')
    print(f'... from {filename} to {extract_dir}')

Extracted 201309-citibike-tripdata.csv
Extracted 201311-citibike-tripdata.csv
Extracted 201307-citibike-tripdata.csv
Extracted 201308-citibike-tripdata.csv
Extracted 201306-citibike-tripdata.csv
Extracted 201310-citibike-tripdata.csv
Extracted 201312-citibike-tripdata.csv
Extracted 201312-citibike-tripdata_1.csv
Extracted 201311-citibike-tripdata_1.csv
Extracted 201307-citibike-tripdata_1.csv
Extracted 201310-citibike-tripdata_2.csv
Extracted 201310-citibike-tripdata_1.csv
Extracted 201309-citibike-tripdata_2.csv
Extracted 201309-citibike-tripdata_1.csv
Extracted 201308-citibike-tripdata_1.csv
Extracted 201308-citibike-tripdata_2.csv
Extracted 201306-citibike-tripdata_1.csv
from 2013-citibike-tripdata.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted 201404-citibike-tripdata_1.csv
Extracted 201412-citibike-tripdata_1.csv
Extracted 201411-citibike-tripdata_1.csv
Extracted 201407-citibike-tripdata_1.csv
Extracted 201410-citibike-tripdata_1.csv
Extracted 20140

Extracted 202407-citibike-tripdata_1.csv
Extracted 202407-citibike-tripdata_2.csv
Extracted 202407-citibike-tripdata_3.csv
Extracted 202407-citibike-tripdata_4.csv
Extracted 202407-citibike-tripdata_5.csv
from 202407-citibike-tripdata.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted 202408-citibike-tripdata_3.csv
Extracted 202408-citibike-tripdata_2.csv
Extracted 202408-citibike-tripdata_1.csv
Extracted 202408-citibike-tripdata_5.csv
Extracted 202408-citibike-tripdata_4.csv
from 202408-citibike-tripdata.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted JC-201509-citibike-tripdata.csv
from JC-201509-citibike-tripdata.csv.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted JC-201510-citibike-tripdata.csv
from JC-201510-citibike-tripdata.csv.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted JC-201511-citibike-tripdata.csv
from JC-201511-citibike-tripdata.csv.zip to C:\Users\H

Extracted JC-202007-citibike-tripdata.csv
from JC-202007-citibike-tripdata.csv.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted JC-202008-citibike-tripdata.csv
from JC-202008-citibike-tripdata.csv.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted JC-202009-citibike-tripdata.csv
from JC-202009-citibike-tripdata.csv.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted JC-202010-citibike-tripdata.csv
from JC-202010-citibike-tripdata.csv.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted JC-202011-citibike-tripdata.csv
from JC-202011-citibike-tripdata.csv.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted JC-202012-citibike-tripdata.csv
from JC-202012-citibike-tripdata.csv.zip to C:\Users\Hanna\sciebo\AXA_coding-challenge/data/bike-tripdata
Extracted JC-202101-citibike-tripdata.csv
from JC-202101-citibike-tripdata.csv.zip to C:\Users\Hanna\sciebo\AXA_

## Visualize dataset and clean
1. Column names:
  * Check which unique column names exist across all files (since they are inconsistent)
  * Correct column names (strip uppercase and convert space to underscore) and create global column name list
2. Dtypes:
  * Check which dtypes files contain, and whether they are consistent across all files (-> they are not)
  * Decide which columns to change to which dtype (to keep cleaned file as small as possible)
4. For categorical data:
  * Check which unique entries exist across files and save these in a dictionary
5. Loop over all csv files, concatenate and save (and deal with some nans)
  * For each file: (in pandas) clean column names, remove unwanted columns, add nans for global columns that don´t exist in individual file, set desired dtype and category dealing with nans, drop duplicate rows, add 'year' and 'month' column from 'start_datetime'; (in dask) convert to dask df for more efficiency and append to big list

## Concatenate files into dask parquet file
6. Concatenate all files, remove duplicates, and save as dask parquet file (partitioned in year and month)

In [12]:
# Since CSV files do not contain the same column headers, check which ones exist in the dataset?
list_files = [f for f in os.listdir(extract_dir) if f.endswith('.csv')]
unique_column_names = []
for csv_file in list_files:
    file_path = os.path.join(extract_dir, csv_file)
    df = pd.read_csv(file_path, nrows=3)
    [unique_column_names.append(col) for col in df.columns if col not in unique_column_names]
print(unique_column_names)

['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual']


It turns out that the column names are not consistent, e.g. some files contain the column "starttime" while others contain the column "Start Time". This should be corrected. Additionally, column names should not contain spaces ("start station latitude" vs "start_lat"). Last, there are 2 strange column names which need to be checked: "Unnamed: 0" and "rideable_type_duplicate_column_name_1".

### 1. Column names

In [13]:
unique_column_names = list_unique_col_names(list_files)
print('Unique column names: \n  ' + str(unique_column_names))
column_mapping = {
    'bikeid': 'bike_id',
    'end_lat': 'end_station_latitude',
    'end_lng': 'end_station_longitude',
    'ended_at': 'end_datetime',
    'member_casual': 'user_type',
    'rideable_type_duplicate_column_name_1': 'duplicate_col',
    'start_lat': 'start_station_latitude',
    'start_lng': 'start_station_longitude',
    'starttime': 'start_datetime',
    'start_time': 'start_datetime',
    'started_at': 'start_datetime',
    'stoptime': 'end_datetime',
    'stop_time': 'end_datetime',
    'tripduration': 'trip_duration',
    'unnamed:_0': 'unnamed', # this is just an index column without name, present in some files -> can be discarded later
    'usertype': 'user_type'
}

# Exploratory correction, see if it solves the inconsistencies
unique_column_names=[]
for csv_file in list_files:
    file_path = os.path.join(extract_dir, csv_file)
    df = pd.read_csv(file_path, nrows=1)
#     if 'rideable_type_duplicate_column_name_1' in df.columns: # check what column this is -> just a duplicate -> can be discarded
#         print(df.head(2))
    df = clean_column_names(df, column_mapping)
    ##print(df.columns)
    
    [unique_column_names.append(col) for col in df.columns if col not in unique_column_names] # save new col names for checking
    unique_column_names.sort()

remove_names = ['unnamed','duplicate_col'] # column names to remove
final_column_names = [name for name in unique_column_names if name not in remove_names] # list with final universal column names
print(' ')
print('Unique column names after cleaning: \n ' + str(final_column_names)) # -> satisfied!
final_column_names.extend(['year','month']) # add the columns year and month, as I will add them from start_datetime

Unique column names: 
  ['end_lat', 'end_lng', 'end_station_id', 'end_station_name', 'ended_at', 'member_casual', 'ride_id', 'rideable_type', 'start_lat', 'start_lng', 'start_station_id', 'start_station_name', 'started_at']
 
Unique column names after cleaning: 
 ['end_datetime', 'end_station_id', 'end_station_latitude', 'end_station_longitude', 'end_station_name', 'ride_id', 'rideable_type', 'start_datetime', 'start_station_id', 'start_station_latitude', 'start_station_longitude', 'start_station_name', 'user_type']


### 2. Dtypes

In [14]:
# check if data type of files are the same
check_dtype = {col: [] for col in final_column_names} # create empty dict to store dtypes

list_files = [f for f in os.listdir(extract_dir) if f.endswith('.csv')]
for csv_file in list_files:
#df.memory_usage(deep=True).sum()
    file_path = os.path.join(extract_dir, csv_file)
    df = pd.read_csv(file_path, nrows=3)
    df = clean_column_names(df, column_mapping) # clean column names
    to_remove = ['duplicate_col','unnamed'] 
    for col in to_remove:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True) # drop these columns
    for col in df.columns:
        #print(f'{col}: {df[col].dtype}')
        check_dtype[col].append(df[col].dtype)

for col in check_dtype:
    print(f"Column: {col}")
    print(f"Unique dtypes: {set(check_dtype[col])}\n")

Column: end_datetime
Unique dtypes: {dtype('O')}

Column: end_station_id
Unique dtypes: {dtype('float64')}

Column: end_station_latitude
Unique dtypes: {dtype('float64')}

Column: end_station_longitude
Unique dtypes: {dtype('float64')}

Column: end_station_name
Unique dtypes: {dtype('O')}

Column: ride_id
Unique dtypes: {dtype('O')}

Column: rideable_type
Unique dtypes: {dtype('O')}

Column: start_datetime
Unique dtypes: {dtype('O')}

Column: start_station_id
Unique dtypes: {dtype('float64')}

Column: start_station_latitude
Unique dtypes: {dtype('float64')}

Column: start_station_longitude
Unique dtypes: {dtype('float64')}

Column: start_station_name
Unique dtypes: {dtype('O')}

Column: user_type
Unique dtypes: {dtype('O')}

Column: year
Unique dtypes: set()

Column: month
Unique dtypes: set()



In [15]:
df.head() # view the last csv file, still loaded in memory

,ride_id,rideable_type,start_datetime,end_datetime,start_station_name,start_station_id,end_station_name,end_station_id,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,user_type
0,5078F3D302000BD2,electric_bike,2024-01-22 18:43:19.012,2024-01-22 18:48:10.708,Frederick Douglass Blvd & W 145 St,7954.1200,St Nicholas Ave & W 126 St,7756.1000,40.8231,-73.9417,40.8114,-73.9519,member
1,814337105D37302A,electric_bike,2024-01-11 19:19:18.721,2024-01-11 19:47:36.007,W 54 St & 6 Ave,6771.1300,E 74 St & 1 Ave,6953.0800,40.7618,-73.9770,40.7690,-73.9548,member
2,A33A920E2B10710C,electric_bike,2024-01-30 19:17:41.693,2024-01-30 19:32:49.857,E 11 St & Ave B,5659.1100,W 10 St & Washington St,5847.0600,40.7276,-73.9798,40.7334,-74.0085,casual


### 4. Categorical data

In [16]:
# Get universal entries for columns I am converting to str and then to category (global categories across csv files)
dtype_cat =['gender', 'user_type', 'rideable_type']

# get global categories (differing per file)
categories_dict = {col: set() for col in dtype_cat }

# Collect all unique categories across the DataFrames
for csv_file in list_files:  
    file_path = os.path.join(extract_dir, csv_file) # load individual file
    
    # Process the file in chunks to save memory
    df = pd.read_csv(file_path)
    df = clean_column_names(df, column_mapping) # clean column names
    to_remove = ['duplicate_col','unnamed'] 
    for col in to_remove:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True) # drop these columns
        
    for col in df.columns:
        if col in categories_dict:
            df[col].fillna('unknown').dropna()   
            categories_dict[col].update(df[col].unique())  # update unique categories of df
            #categories_dict[col].add('unknown') # add category 'unknown' for missing data
categories_dict['user_type'].discard(np.nan) # for some reason still nan as category here
categories_dict

C:\Users\Hanna\AppData\Local\Temp\ipykernel_24200\3511905405.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


{'gender': set(),
 'user_type': {'casual', 'member'},
 'rideable_type': {'classic_bike', 'electric_bike'}}

### 5. & 6. Loop over all csv files, concatenate and save as dask parquet file

In [3]:
### This cell executed in the Anaconda powershell (clean_concat.py), since it´s faster/requires less memory on my 16GB mem laptop ###

if not os.path.exists(cleaned_dir):  # directory to save the cleaned df
    os.makedirs(cleaned_dir)

dtype_dict = { # desired dtypes
    'int32': ['birth_year', 'trip_duration'],
    'float32': ['start_station_id', 'end_station_id'], 
    'float64': ['start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude'],
    'str': ['start_station_name', 'end_station_name', 'ride_id'], 
    'category': ['gender', 'user_type', 'rideable_type'],
    'datetime64[ns]': ['start_datetime', 'end_datetime']
}

dtype_mapping = {} # dictionary with col: dtype, for changing data types per column
for dtype, columns in dtype_dict.items():
    for col in columns:
        dtype_mapping[col] = dtype

list_files = [f for f in os.listdir(extract_dir) if f.endswith('.csv')] # list of csv files in dir to loop over
chunksize = 1_000_000  # load in chunks to save memory, in case csv file is huge

ddf_list = []
for csv_file in list_files[0:2]: 
    file_path = os.path.join(extract_dir, csv_file) # load individual file
    print('loading ' +  csv_file)
    
    # Process the file in chunks to save memory
    chunk_iter = pd.read_csv(file_path, chunksize=chunksize, low_memory=True, parse_dates=True)

    for n, chunk in enumerate(chunk_iter):
        ddf = clean_column_names(chunk, column_mapping)  # Clean column names
 
        # Convert to Dask DataFrame for larger datasets
        #ddf = dd.from_pandas(chunk, npartitions=1) #
        #print('to dask converted')
        # Drop unwanted columns
        to_remove = ['duplicate_col', 'unnamed']
        ddf = ddf.drop(columns=[col for col in to_remove if col in ddf.columns])
        
        missing_cols = set(final_column_names) - set(ddf.columns) # add missing (universal) columns from final_column_names and fill with nans
        for col in missing_cols:
            ddf[col] = np.nan
                
        # Convert column dtypes
        for col in ddf.columns:
            if col in dtype_dict['int32']:
                ddf[col] = ddf[col].replace('\\N', np.nan)  # handle missing values
                ddf[col] = pd.to_numeric(ddf[col], errors='coerce').astype('float32')
                ddf[col] = ddf[col].fillna(0).round(0).astype('int32') # replace nan with the place filler 0, round and convert to int
            elif col in dtype_dict['category']: # for categorical data, replace nans with 'unknown' cat
                ddf[col] = ddf[col].fillna('unknown')  # Replace NaNs with 'unknown'
                ddf[col] = ddf[col].astype('str') # convert to str first
                ddf[col] = ddf[col].astype('category') # string to category, as it needs less memory
                ddf[col] = ddf[col].cat.set_categories(new_categories=list(categories_dict[col])) # set global categories
            elif col in dtype_dict['str']: # - added 6.10
                ddf[col] = ddf[col].fillna('unknown')  # Replace NaNs with 'unknown'
                ddf[col] = ddf[col].astype(dtype_mapping[col]) # 
            elif col in dtype_mapping.keys():
                if col in dtype_dict['float32'] or col in col in dtype_dict['float64']:
                    ddf[col] = pd.to_numeric(ddf[col], errors='coerce')
                ddf[col] = ddf[col].astype(dtype_mapping[col]) 
                
        ddf = ddf.drop_duplicates().sort_values(by='start_datetime')# drop duplicates and sort to start rental time/date
        ddf = ddf.reset_index(drop=True)
        #ddf = ddf.set_index('start_datetime') # set start_datetime as index

        ddf['year'] = ddf['start_datetime'].dt.year.astype('int32') # add year column for partitioning
        ddf['month'] = ddf['start_datetime'].dt.month.astype('int8') # add month column for partitioning
                                            
        ddf = ddf[final_column_names] # ensure consistent column order   

        ddf = dd.from_pandas(ddf, npartitions=5)
        ddf_list.append(ddf)   
    print('.... cleaned, converted to dask df and appended to ddf_list')     

ddf_comb = dd.concat(ddf_list, ignore_index=True) # concatenate all dask dfs
#del ddf_list
ddf_comb = ddf_comb.drop_duplicates() # remove duplicates

# check if all dtypes are consistent across ddfs
check_dtype = {col: [] for col in final_column_names}
for ddf in ddf_list:
    for col in ddf.columns:
        check_dtype[col].append(ddf[col].dtype)

for col in check_dtype.keys():
    print(col)
    print(pd.Series(check_dtype[col]).unique())
    if len(pd.Series(check_dtype[col]).unique()) > 1:
        print(col + ' has inconsistent dtypes')
    
ddf_comb.to_parquet(cleaned_dir + '/combined_dask_df.parquet', engine='pyarrow', partition_on=['year', 'month'], write_index=False) # write, partitioned on year and month
print('All files processed and saved to Parquet')

## Load cleaned ddf (data bike rides)
Some additional cleaning:
* Set categorical entries consistently (user_type)
* More dtype changes after inspecting the cleaned data

In [3]:
# Load cleaned data from saved file
ddf = dd.read_parquet(cleaned_dir + '/bike-tripdata_cleaned/combined_dask_df_0910.parquet')
#ddf = dd.read_parquet('path_to_parquet_file', columns=['category_column', 'numeric_column']) # read only specific columns
#ddf_2020 = dd.read_parquet('path_to_parquet_file/year=2020') # read only specific partition

ddf.head()

,bike_id,birth_year,end_datetime,end_station_id,end_station_latitude,end_station_longitude,end_station_name,gender,ride_id,rideable_type,start_datetime,start_station_id,start_station_latitude,start_station_longitude,start_station_name,trip_duration,user_type,year,month
0,16449.0,1959,2013-06-01 10:14:22,303.0,40.723627,-73.999496,None,NaN,None,NaN,2013-06-01 09:59:58,376.0,40.708621,-74.007222,None,864,NaN,2013,6
1,19370.0,0,2013-06-01 13:18:16,466.0,40.743954,-73.991449,None,NaN,None,NaN,2013-06-01 12:53:25,530.0,40.771522,-73.990541,None,1491,NaN,2013,6
2,17565.0,1986,2013-06-01 14:48:57,325.0,40.736245,-73.984738,None,NaN,None,NaN,2013-06-01 14:40:05,382.0,40.734927,-73.992005,None,532,NaN,2013,6
3,19200.0,0,2013-06-01 20:35:56,468.0,40.765265,-73.981923,None,NaN,None,NaN,2013-06-01 20:04:40,448.0,40.756604,-73.997901,None,1876,NaN,2013,6
4,14784.0,1984,2013-06-01 21:49:47,514.0,40.760875,-74.002777,None,NaN,None,NaN,2013-06-01 21:37:24,470.0,40.743453,-74.000040,None,743,NaN,2013,6


Now that all column names are consistent and all data is concatenated, check if categories in some columns are consistent

In [5]:
print('Unique categories:')
print(f'user_type - {ddf["user_type"].cat.as_known().cat.categories}')
print(f'rideable_type - {ddf["rideable_type"].cat.as_known().cat.categories}')

print('Unique string:')
print('start_station id, unique: '+str(ddf.start_station_id.nunique().compute()))
print('start_station name, unique: '+str(ddf.start_station_name.nunique().compute()))
print('end_station_id, unique: '+str(ddf.end_station_id.nunique().compute()))
print('end_station name, unique: '+str(ddf.end_station_name.nunique().compute()))

Unique categories:
user_type - Index(['member', 'Customer', 'Subscriber', 'casual'], dtype='object')
rideable_type - Index(['electric_bike', 'classic_bike', 'docked_bike'], dtype='object')
Unique string:
start_station id, unique: 3318
start_station name, unique: 2581
end_station_id, unique: 3359
end_station name, unique: 2610


In [6]:
print(ddf.dtypes)

bike_id                            float64
birth_year                           int32
end_datetime                datetime64[ns]
end_station_id                     float32
end_station_latitude               float64
end_station_longitude              float64
end_station_name           string[pyarrow]
gender                            category
ride_id                    string[pyarrow]
rideable_type                     category
start_datetime              datetime64[ns]
start_station_id                   float32
start_station_latitude             float64
start_station_longitude            float64
start_station_name         string[pyarrow]
trip_duration                        int32
user_type                         category
year                              category
month                             category
dtype: object


In [4]:
# Check memory usage of ddf
memory_usage = ddf.memory_usage(deep=True).compute()
total_memory_usage = memory_usage.sum() # total memory usage in bytes
print(f"Total memory usage of ddf: {total_memory_usage / (1024**3):.2f} GB") # convert to GB

Total memory usage of ddf: 17.28 GB


Okay, this shows that the ddf still requires a lot of memory, therefore I will see if I can further reduce this (by changing to the most efficient dtype, and rounding floats).

In [10]:
# Additional adjustments of dtype - check min and max, to decide which dtype can represent all values

print(f"bike_id (min, max): {ddf['bike_id'].dropna().min().compute()}, {ddf['bike_id'].dropna().max().compute()}")
print(f"start_station_id (min, max): {ddf['start_station_id'].dropna().min().compute()}, {ddf['start_station_id'].dropna().max().compute()}")
print(f"end_station_id (min, max): {ddf['end_station_id'].dropna().min().compute()}, {ddf['end_station_id'].dropna().max().compute()}")
print(f"trip_duration (min, max): {ddf['trip_duration'].dropna().min().compute()}, {ddf['trip_duration'].dropna().max().compute()}")

bike_id (min, max): 14529.0, 49985.0
start_station_id (min, max): 72.0, 8897.0498046875
end_station_id (min, max): 72.0, 8897.0498046875
trip_duration (min, max): 0, 20260212


In [12]:
ddf['start_station_id'].unique().compute() # -> decimals, I didnt expect that

0   3206.0000
0   8451.0703
0   3383.0000
0   6889.1201
0   4122.0298
       ...   
0   6190.0298
0   8485.0098
0   3900.0000
1   7820.0498
0   8472.0596
Name: start_station_id, Length: 3319, dtype: float32

In [4]:
# Additional adjustments of dtype, based on min and max value in the dataset

# bike_id -> overlooked. Ideally, do it in the previous cleaning step, but since that takes a long time to run and I don´t have much time left, I´m doing it here for now
ddf['bike_id'] = pd.to_numeric(ddf['bike_id'], errors='coerce')
ddf['bike_id'] = ddf['bike_id'].round(0).astype('Int64')  

# start_station_id and end_station_id -> Int64 which can handle NaNs (I didn´t know earlier), and decimals are not needed
ddf['start_station_id'] = ddf['start_station_id'].round(0).astype('Int64')
ddf['end_station_id'] = ddf['end_station_id'].round(0).astype('Int64')

# birth_year can be int16 (range: -32768 to 32767) as NaNs are set to 0 already
ddf['birth_year'] = ddf['birth_year'].astype('int16')
ddf['trip_duration'] = ddf['trip_duration'].astype('int32') #

ddf = ddf.sort_values(by='start_datetime')# sort again to start rental time/date, since ddf does not seem sorted (time starts at 10am instead of midnight)

#ddf.compute()
ddf.head(2)

,bike_id,birth_year,end_datetime,end_station_id,end_station_latitude,end_station_longitude,end_station_name,gender,ride_id,rideable_type,start_datetime,start_station_id,start_station_latitude,start_station_longitude,start_station_name,trip_duration,user_type,year,month
0,19678,1983,2013-06-01 00:11:36,434,40.743174,-74.003664,None,NaN,None,NaN,2013-06-01 00:00:01,444,40.742354,-73.989151,None,695,NaN,2013,6
0,16649,1984,2013-06-01 00:11:41,434,40.743174,-74.003664,None,NaN,None,NaN,2013-06-01 00:00:08,444,40.742354,-73.989151,None,693,NaN,2013,6


In [5]:
# Unify string in rideable_type and user_type columns

#ddf['rideable_type'] = ddf['rideable_type'].astype('str').str.strip().str.lower().astype('category')# if needed

def replace_user_type(df):
    df = df.copy()  # make a copy to avoid SettingWithCopyWarning
    df['user_type'] = df['user_type'].astype(str).str.strip().str.lower() # temporarily convert to string (object)
    
    # Replace 'subscriber' with 'member' and 'customer' with 'casual'
    df.loc[df['user_type'] == 'subscriber', 'user_type'] = 'member'
    df.loc[df['user_type'] == 'customer', 'user_type'] = 'casual'

    df['user_type'] = pd.Categorical(df['user_type'], categories=['member', 'casual']) # convert back to category

    return df

ddf = ddf.map_partitions(replace_user_type)

print('Unique categories, after changing "subscriber" to "member" and "customer" to "casual", for consistency:')
print(f'user_type - {ddf["user_type"].cat.as_known().cat.categories}')
ddf.head(2) # check if it worked

Unique categories, after changing "subscriber" to "member" and "customer" to "casual", for consistency:
user_type - Index(['member', 'casual'], dtype='object')


,bike_id,birth_year,end_datetime,end_station_id,end_station_latitude,end_station_longitude,end_station_name,gender,ride_id,rideable_type,start_datetime,start_station_id,start_station_latitude,start_station_longitude,start_station_name,trip_duration,user_type,year,month
0,19678,1983,2013-06-01 00:11:36,434,40.743174,-74.003664,None,NaN,None,NaN,2013-06-01 00:00:01,444,40.742354,-73.989151,None,695,NaN,2013,6
0,16649,1984,2013-06-01 00:11:41,434,40.743174,-74.003664,None,NaN,None,NaN,2013-06-01 00:00:08,444,40.742354,-73.989151,None,693,NaN,2013,6


In [ ]:
ddf.isna().sum().compute() # check how many nans in which columns -- taking too long

In [ ]:
# remove rows when nans in important columns
# important_col = ['end_datetime','start_datetime','end_station_latitude','start_station_latitude',
#                  'gender','birth_year','rideable_type','trip_duration','user_type']
# ddf.dropna(subset=important_col, inplace=True) # drop all rows where entry is NaN in important columns (that I will use)

In [ ]:
#ddf[ddf['end_station_latitude'].isna()].compute().head(5)

In [ ]:
# # Get some idea of the df content

# print('Mean trip duration: ')
# print(ddf.groupby('user_type').agg({'trip_duration': 'mean'}).compute())
# ddf.groupby('gender').agg({'trip_duration': 'mean'}).compute()

In [ ]:
# save further cleaned ddf again, overwriting
ddf.to_parquet(cleaned_dir + '/combined_dask_df_cleaned.parquet', engine='pyarrow', partition_on=['year', 'month'], write_index=False) # write, partitioned on year and month
print('ddf saved to parquet')

In [ ]:
# If dataset is too big for the code to run smoothly you can save a random sample here
df_sample = ddf.sample(frac=0.01)
df_sample.to_parquet(os.path.join(cleaned_dir,'df_rides_1010_sampled1%.parquet'), engine='pyarrow', write_index=False)
print('ddf saved to sampled1%')
df_sample = ddf.sample(frac=0.001)
df_sample.to_parquet(os.path.join(cleaned_dir,'df_rides_1010_sampled0_1%.parquet'), engine='pyarrow', write_index=False)
print('ddf saved to sampled0.1%')
df_sample = ddf.sample(frac=0.0001)
df_sample.to_parquet(os.path.join(cleaned_dir,'df_rides_1010_sampled0_01%.parquet'), engine='pyarrow', write_index=False)
print('ddf saved to sampled0.001%')
df_sample = ddf.sample(frac=0.1)
df_sample.to_parquet(os.path.join(cleaned_dir,'df_rides_1010_sampled10%.parquet'), engine='pyarrow', partition_on=['year'], write_index=False)
print('ddf saved to sampled10%')

In [ ]:
# Check memory usage of ddf after additional data cleaning
total_memory_usage = ddf.memory_usage(deep=True).sum().compute() # total memory usage in bytes
print(f"Total memory usage of ddf after additional data cleaning: {total_memory_usage / (1024**3):.2f} GB") # convert to GB

## Collision data
### 1. Data inspection
### 2. Cleaning: column names, entries, dtype, missing data
### 3. Add column 'bike_involved' to access bike accident
### 4. Save cleaned data as csv file

In [ ]:
file_path = current_dir + '/raw_data/Motor_Vehicle_Collisions_-_Crashes_20240922.csv' # load file
df = pd.read_csv(file_path)

### 1. Data inspection

In [18]:
print(df.shape)
df.head(5)

(2120518, 29)


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


### 2. Cleaning: column names, entries, dtype, missing data

In [19]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_') # clean column names
print(df.columns)
print(df.dtypes)

# How many missing values (nans) and where
summary_table = pd.DataFrame({
    'Nan_count': df.isna().sum(),
    'Total': df.shape[0]
})
print(summary_table)

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'latitude',
       'longitude', 'location', 'on_street_name', 'cross_street_name',
       'off_street_name', 'number_of_persons_injured',
       'number_of_persons_killed', 'number_of_pedestrians_injured',
       'number_of_pedestrians_killed', 'number_of_cyclist_injured',
       'number_of_cyclist_killed', 'number_of_motorist_injured',
       'number_of_motorist_killed', 'contributing_factor_vehicle_1',
       'contributing_factor_vehicle_2', 'contributing_factor_vehicle_3',
       'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5',
       'collision_id', 'vehicle_type_code_1', 'vehicle_type_code_2',
       'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5'],
      dtype='object')
crash_date                        object
crash_time                        object
borough                           object
zip_code                          object
latitude                         float64
longitude      

In [20]:
# Correct the columns number_of_persons_injured and number_of_persons_killed, that contain nan
print(df.loc[df['number_of_persons_injured'].isna(),['number_of_persons_injured','number_of_pedestrians_injured','number_of_cyclist_injured',
                                                'number_of_motorist_injured']].head()) # show 

df.loc[df['number_of_persons_injured'].isna(), 'number_of_persons_injured'] = df['number_of_pedestrians_injured'] + df['number_of_cyclist_injured'] + df['number_of_motorist_injured']
df.loc[df['number_of_persons_killed'].isna(), 'number_of_persons_killed'] = df['number_of_pedestrians_killed'] + df['number_of_cyclist_killed'] + df['number_of_motorist_killed']
                                                                               
df[df['number_of_persons_injured'].isna()] # show again after correction                                      

        number_of_persons_injured  number_of_pedestrians_injured  \
182614                        NaN                              0   
569936                        NaN                              0   
619341                        NaN                              0   
669416                        NaN                              0   
712527                        NaN                              0   

        number_of_cyclist_injured  number_of_motorist_injured  
182614                          1                           0  
569936                          0                           1  
619341                          0                           1  
669416                          0                           0  
712527                          0                           1  


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5


In [21]:
# change column names 'longitude' and 'latitude' for later combining with the bike ride dataset, and number_of_/cyclist/motorist to avoid confusion
df = df.rename(columns={'latitude': 'accident_latitude', 'longitude': 'accident_longitude', 'number_of_cyclist_injured':'number_of_cyclists_injured',
                        'number_of_cyclist_killed':'number_of_cyclists_killed','number_of_motorist_injured':'number_of_motorists_injured',
                       'number_of_motorist_killed':'number_of_motorists_killed'})

In [11]:
# Check format of entries in columns
print(df['zip_code'].unique()[0:10])
print(df['number_of_persons_injured'].unique())
df['number_of_persons_killed'].unique()

[nan 11208.0 11233.0 10475.0 11207.0 10017.0 11413.0 11434.0 11217.0
 11226.0]
[ 2.  1.  0.  4.  3.  5.  7.  6.  9. 18.  8. 11. 17. 10. 14. 15. 12. 13.
 40. 16. 20. 22. 31. 19. 27. 32. 24. 43. 21. 23. 34. 25.]


array([0., 1., 2., 3., 4., 8., 5.])

In [22]:
dtype_dict = { 
    'Int64': ['zip_code'],
    'float64': ['accident_latitude', 'accident_longitude'],
    'str': ['borough','on_street_name','cross_street_name','off_street_name','contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
           'contributing_factor_vehicle_3','contributing_factor_vehicle_4','contributing_factor_vehicle_5','vehicle_type_code_1',
           'vehicle_type_code_2','vehicle_type_code_3','vehicle_type_code_4','vehicle_type_code_5'], 
    'int8': ['number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed', 
             'number_of_cyclists_injured', 'number_of_cyclists_killed', 'number_of_motorists_injured', 'number_of_motorists_killed'],
    'datetime64[ns]' : ['crash_datetime']
}

dtype_mapping = {}
for dtype, columns in dtype_dict.items():
    for col in columns:
        dtype_mapping[col] = dtype

for col in df.columns:
    if col not in dtype_mapping.keys():
        continue
    elif col == 'zip_code': # can´t convert to int directly because it has string with empty entries
        df[col] = pd.to_numeric(df[col].str.strip(), errors='coerce').astype('float32')
        df[col] = df[col].round(0).astype('Int64');
    elif col in dtype_dict['str']: # for categorical data, replace nans with 'unknown' cat
        df[col].fillna('unknown', inplace=True)
        df[col] = df[col].astype(dtype_mapping[col])
        df[col]  = df[col].str.strip().str.lower() # clean string entries
        # df[col] = df[col].astype('category') # string to category, as it needs less memory
        # df[col] = df[col].cat.add_categories('unknown').fillna('unknown') # add unknown category for nans
    elif col in dtype_dict['int8']:
        df[col] = df[col].astype(dtype_mapping[col])
        df[col].round(0)
    else: 
        df[col] = df[col].astype(dtype_mapping[col])
        df[col].fillna(np.nan, inplace=True)
            
df['crash_datetime'] = pd.to_datetime(df['crash_date'] + ' ' + df['crash_time']) #
df.drop(['crash_date','crash_time'], axis=1, inplace=True)
df['year'] = df['crash_datetime'].dt.year.astype('int32') # add year column 
df['month'] = df['crash_datetime'].dt.month.astype('int8') # add month column 

dupl_before =df.shape[0]
df = df.drop_duplicates().sort_values(by='crash_datetime')# drop duplicates and sort to start rental time/date
print(f'{dupl_before - df.shape[0]} duplicates removed')
df = df.reset_index(drop=True)

print(' Unique values in zip_code: '); print(df.zip_code.unique())
df.head() # show cleaned data

0 duplicates removed
 Unique values in zip_code: 
<IntegerArray>
[ <NA>, 10036, 11223, 11215, 10451, 11234, 11375, 11233, 10007, 10017,
 ...
 10281, 11695, 10282, 11359, 10111, 10803, 10153, 10048, 10168, 10162]
Length: 194, dtype: Int64


,borough,zip_code,accident_latitude,accident_longitude,location,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,...,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,crash_datetime,year,month
0,unknown,<NA>,40.697753,-73.813916,"(40.6977532, -73.8139159)",unknown,unknown,unknown,1,0,...,unknown,2999940,passenger vehicle,passenger vehicle,unknown,unknown,unknown,2012-07-01 00:05:00,2012,7
1,manhattan,10036,40.762127,-73.997387,"(40.7621266, -73.9973865)",11 avenue,west 44 street,unknown,0,0,...,unknown,37632,passenger vehicle,bus,unknown,unknown,unknown,2012-07-01 00:05:00,2012,7
2,brooklyn,11223,40.588868,-73.972745,"(40.5888678, -73.9727446)",west 3 street,bouck court,unknown,0,0,...,unknown,116256,passenger vehicle,sport utility / station wagon,unknown,unknown,unknown,2012-07-01 00:10:00,2012,7
3,unknown,<NA>,40.733610,-73.923840,"(40.73361, -73.9238405)",unknown,unknown,unknown,1,0,...,unknown,3044659,passenger vehicle,passenger vehicle,passenger vehicle,passenger vehicle,unknown,2012-07-01 00:10:00,2012,7
4,brooklyn,11215,40.677406,-73.983048,"(40.6774056, -73.9830482)",4 avenue,union street,unknown,0,0,...,unknown,175808,unknown,bicycle,unknown,unknown,unknown,2012-07-01 00:20:00,2012,7


In [23]:
# Check if df still has missing values (nans) and how many
summary_table = pd.DataFrame({
    'Nan_count': df.isna().sum(),
    'Total': df.shape[0]
})

print(summary_table)

                               Nan_count    Total
borough                                0  2120518
zip_code                         1721072  2120518
accident_latitude                 247820  2120518
accident_longitude                247820  2120518
location                          247820  2120518
on_street_name                         0  2120518
cross_street_name                      0  2120518
off_street_name                        0  2120518
number_of_persons_injured              0  2120518
number_of_persons_killed               0  2120518
number_of_pedestrians_injured          0  2120518
number_of_pedestrians_killed           0  2120518
number_of_cyclists_injured             0  2120518
number_of_cyclists_killed              0  2120518
number_of_motorists_injured            0  2120518
number_of_motorists_killed             0  2120518
contributing_factor_vehicle_1          0  2120518
contributing_factor_vehicle_2          0  2120518
contributing_factor_vehicle_3          0  2120518


Missing data in string columns handled well -> all replaced with 'unknown', no nans left

In [10]:
df.describe()

,zip_code,accident_latitude,accident_longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclists_injured,number_of_cyclists_killed,number_of_motorists_injured,number_of_motorists_killed,collision_id,crash_datetime,year,month
count,399446.0,1.872698e+06,1.872698e+06,2.120518e+06,2.120518e+06,2.120518e+06,2.120518e+06,2.120518e+06,2.120518e+06,2.120518e+06,2.120518e+06,2.120518e+06,2120518,2.120518e+06,2.120518e+06
mean,10801.207059,4.062109e+01,-7.374009e+01,3.160567e-01,1.526042e-03,5.728270e-02,7.531179e-04,2.758383e-02,1.188389e-04,2.271728e-01,6.305063e-04,3.193116e+06,2017-09-11 06:15:02.005152512,2.017182e+03,6.671396e+00
min,10000.0,0.000000e+00,-2.013600e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.200000e+01,2012-07-01 00:05:00,2.012000e+03,1.000000e+00
25%,10065.0,4.066767e+01,-7.397478e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.166249e+06,2015-02-07 09:15:45,2.015000e+03,4.000000e+00
50%,11204.0,4.072065e+01,-7.392715e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.696518e+06,2017-06-16 16:00:00,2.017000e+03,7.000000e+00
75%,11235.0,4.076963e+01,-7.386673e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.226882e+06,2019-10-19 06:29:30,2.019000e+03,1.000000e+01
max,11697.0,4.334444e+01,0.000000e+00,4.300000e+01,8.000000e+00,2.700000e+01,6.000000e+00,4.000000e+00,2.000000e+00,4.300000e+01,5.000000e+00,4.757303e+06,2024-09-17 23:45:00,2.024000e+03,1.200000e+01
std,568.096756,2.046708e+00,3.837215e+00,7.055802e-01,4.124354e-02,2.455219e-01,2.801105e-02,1.658659e-01,1.094386e-02,6.668334e-01,2.745294e-02,1.506296e+06,NaN,3.203649e+00,3.397637e+00


### 3. Add column 'bike_involved' to access bike accidents

Check how many collisions involved a bike; list all entries that contain 'bi', as probably the entries are inconsistent, 
and they could be 'bike','bicycle' or 'e-bike'

In [11]:
for col in ['vehicle_type_code_1','vehicle_type_code_2','vehicle_type_code_3','vehicle_type_code_4']:
    bike_entries = df[df[col].str.contains('bi', case=False, na=False)]
    print(col); print(bike_entries[col].unique())

vehicle_type_code_1
['bicycle' 'bike' 'motorbike' 'e bik' 'minibike' 'ebike' 'mobil' 'e- bi'
 'e-bik' 'snowmobile' 'cabin' 'bicyc' 'e-bike' 'dirt bike' 'e bike'
 'e bike uni' 'e bike w p' 'combinatio' 'liabitiy' 'pedal bike'
 'mobility s']
vehicle_type_code_2
['bicycle' 'bike' 'motorbike' 'minibike' 'e bik' 'ebike' 'e-bik'
 'snowmobile' 'mobil' 'big r' 'e/bik' 'e-bike' 'mobile foo' 'dirt bike'
 'mobile' 'dirtbike' 'uni e-bike' 'e bike' 'gas bicycl' 'ambiance'
 'dart bike' 'moped bike' 'gas bike' 'mobility s' 'citibike' 'e bike w p'
 'scooter bi']
vehicle_type_code_3
['bicycle' 'bike' 'motorbike' 'e-bik' 'e-bike' 'dirt bike']
vehicle_type_code_4
['bicycle' 'bike' 'e-bike' 'snowmobile' 'motorbike']


As we can see, a lot of different words are used to describe 'bike', with many spelling mistakes. To filter out all bikes, we can select all entries containing 'bik' or 'bic' but exclude 'motorbike'. I will now correct these entries and create an additional column with 'bike'/'no_bike', when any of these 4 columns contain a bike

In [24]:
# Create a mask that checks for 'bik' or 'bic' in any of the 4 columns, excluding 'motorbike'
df['bike_involved'] = 'no_bike'
# Create a mask that checks for 'bik' or 'bic' in any of the 4 columns, excluding 'motorbike'
mask = (
    df[['vehicle_type_code_1', 'vehicle_type_code_2', 'vehicle_type_code_3', 'vehicle_type_code_4']]
    .apply(lambda col: col.str.contains('bik|bic', case=False, na=False))
    .any(axis=1)  # Check if any column contains 'bik' or 'bic'
) & (
    ~df[['vehicle_type_code_1', 'vehicle_type_code_2', 'vehicle_type_code_3', 'vehicle_type_code_4']]
    .apply(lambda col: col.str.contains('motorbike', case=False, na=False))
    .any(axis=1)  # Exclude rows where 'motorbike' appears
)

# Set 'bike_involved' to 'bike' where the condition is met
df.loc[mask, 'bike_involved'] = 'bike'

print(df['bike_involved'].unique())
print(df.loc[df['bike_involved'] == 'no_bike',['bike_involved','vehicle_type_code_1', 'vehicle_type_code_2', 'vehicle_type_code_3', 'vehicle_type_code_4']].head(3))
print(df.loc[df['bike_involved'] == 'bike',['bike_involved','vehicle_type_code_1', 'vehicle_type_code_2', 'vehicle_type_code_3', 'vehicle_type_code_4']].head(3))

['no_bike' 'bike']
  bike_involved vehicle_type_code_1            vehicle_type_code_2  \
0       no_bike   passenger vehicle              passenger vehicle   
1       no_bike   passenger vehicle                            bus   
2       no_bike   passenger vehicle  sport utility / station wagon   

  vehicle_type_code_3 vehicle_type_code_4  
0             unknown             unknown  
1             unknown             unknown  
2             unknown             unknown  
   bike_involved vehicle_type_code_1 vehicle_type_code_2 vehicle_type_code_3  \
4           bike             unknown             bicycle             unknown   
42          bike                taxi             bicycle             unknown   
98          bike   passenger vehicle             bicycle             unknown   

   vehicle_type_code_4  
4              unknown  
42             unknown  
98             unknown  


In [25]:
print(df.dtypes)

borough                                  object
zip_code                                  Int64
accident_latitude                       float64
accident_longitude                      float64
location                                 object
on_street_name                           object
cross_street_name                        object
off_street_name                          object
number_of_persons_injured                  int8
number_of_persons_killed                   int8
number_of_pedestrians_injured              int8
number_of_pedestrians_killed               int8
number_of_cyclists_injured                 int8
number_of_cyclists_killed                  int8
number_of_motorists_injured                int8
number_of_motorists_killed                 int8
contributing_factor_vehicle_1            object
contributing_factor_vehicle_2            object
contributing_factor_vehicle_3            object
contributing_factor_vehicle_4            object
contributing_factor_vehicle_5           

### 4. Save cleaned data as csv file

In [26]:
df.to_csv(cleaned_dir + '/collisions_cleaned.csv') # save cleaned version